In [22]:
%load_ext autoreload
%autoreload 2

In [50]:
import json
import os
import time
import numpy as np
import pandas as pd
from datetime import datetime
from here_location_services import LS
from here_location_services.config.routing_config import ROUTING_RETURN
from here_los_utils import *

In [2]:
LS_API_KEY = os.environ.get("HERE_API_KEY")  # Get API KEY from environment.
ls = LS(api_key=LS_API_KEY)

### Load trips data

In [3]:
trips = pd.read_csv("../data/processed/trip_od_full.csv", low_memory=False)
trips.shape

(36287, 111)

In [4]:
trips.head()

,TRIPID,PERSID,TRIPNO,TRAVDOW,LINKMODE,TRIPPURP,DESTPURP1,STARTHOUR,STARTIME,ARRHOUR,...,OTHERVEHS,TOTALVEHS,TRAVDATE,TRAVMONTH,TRAVYEAR,HOMETAZ,YOUNGEST,AVEAGE,OLDEST,HHINC
0,Y12H0000107P01T01,Y12H0000107P01,1,Sunday,Vehicle Driver,Social,Social,10,630,10,...,0,1,27/05/2012,May,2012,2653,9,21,41,1750
1,Y12H0000107P01T02,Y12H0000107P01,2,Sunday,Vehicle Driver,Social,At or Go Home,12,750,12,...,0,1,27/05/2012,May,2012,2653,9,21,41,1750
2,Y12H0000107P01T03,Y12H0000107P01,3,Sunday,Walking,Recreational,Recreational,13,810,14,...,0,1,27/05/2012,May,2012,2653,9,21,41,1750
3,Y12H0000107P01T04,Y12H0000107P01,4,Sunday,Walking,Recreational,At or Go Home,14,855,14,...,0,1,27/05/2012,May,2012,2653,9,21,41,1750
4,Y12H0000107P02T01,Y12H0000107P02,1,Sunday,Vehicle Passenger,Social,Social,10,630,10,...,0,1,27/05/2012,May,2012,2653,9,21,41,1750


In [5]:
list(trips.columns)

['TRIPID',
 'PERSID',
 'TRIPNO',
 'TRAVDOW',
 'LINKMODE',
 'TRIPPURP',
 'DESTPURP1',
 'STARTHOUR',
 'STARTIME',
 'ARRHOUR',
 'ARRTIME',
 'TRIPTIME',
 'TRAVTIME',
 'WAITIME',
 'ORIGSA1',
 'ORIGSA3',
 'DESTSA1',
 'DESTSA3',
 'CUMDIST',
 'Mode',
 'ORIG_CENTROID',
 'DEST_CENTROID',
 'ORIG_X',
 'ORIG_Y',
 'DEST_X',
 'DEST_Y',
 'HHID',
 'PERSNO',
 'WEEK',
 'MONTHOFBIRTH',
 'YEAROFBIRTH',
 'AGE',
 'AGEGROUP',
 'SEX',
 'RELATIONSHIP',
 'CARLICENCE',
 'MBIKELICENCE',
 'OTHERLICENCE',
 'NOLICENCE',
 'FULLTIMEWORK',
 'PARTTIMEWORK',
 'CASUALWORK',
 'ANYWORK',
 'STUDYING',
 'ACTIVITIES',
 'MAINACT',
 'WORKTYPE',
 'EMPTYPE',
 'ANZSCO1',
 'ANZSIC1',
 'STARTPLACE',
 'ANYSTOPS',
 'NUMSTOPS',
 'REASONCODE',
 'MRTDOW',
 'MRTINT',
 'PERSINC',
 'WHOFILLED',
 'PROXY',
 'FILLDOW',
 'FILLLAG',
 'ADDITIONALTRAVEL',
 'NEWSTOP',
 'DISABLEDTRANS',
 'TAXITRANS',
 'WHEELCHAIRTRANS',
 'ACCESSTRANS',
 'NONETRANS',
 'NOTTOSAYTRANS',
 'CYCLEDWORK',
 'CYCLEDSHOPPING',
 'CYCLEDEXERCISE',
 'CYCLEDOTHER',
 'NOCYCLED',
 'H

In [6]:
cols = ['TRIPID', 'Mode', 'LINKMODE', 'TRIPPURP', 'DESTPURP1', 'TRIPTIME', 'TRAVTIME',
        'TRAVDOW', 'TRAVDATE', 'TRAVMONTH', 'TRAVYEAR', 'STARTHOUR', 'STARTIME', 
        'HomeSA1', 'CUMDIST','ORIGSA1', 'DESTSA1', 'ORIG_X', 'ORIG_Y', 'DEST_X', 'DEST_Y']
trips = trips[cols]
trips.shape

(36287, 21)

In [7]:
trips.head()

,TRIPID,Mode,LINKMODE,TRIPPURP,DESTPURP1,TRIPTIME,TRAVTIME,TRAVDOW,TRAVDATE,TRAVMONTH,...,STARTHOUR,STARTIME,HomeSA1,CUMDIST,ORIGSA1,DESTSA1,ORIG_X,ORIG_Y,DEST_X,DEST_Y
0,Y12H0000107P01T01,Car,Vehicle Driver,Social,Social,20,20,Sunday,27/05/2012,May,...,10,630,20904122028,2.60,20904122028,20904122002,145.087302,-37.659741,145.071523,-37.651478
1,Y12H0000107P01T02,Car,Vehicle Driver,Social,At or Go Home,10,10,Sunday,27/05/2012,May,...,12,750,20904122028,2.60,20904122002,20904122028,145.071523,-37.651478,145.087302,-37.659741
2,Y12H0000107P01T03,Walk,Walking,Recreational,Recreational,30,30,Sunday,27/05/2012,May,...,13,810,20904122028,3.23,20904122028,20904122031,145.087302,-37.659741,145.062231,-37.664970
3,Y12H0000107P01T04,Walk,Walking,Recreational,At or Go Home,30,30,Sunday,27/05/2012,May,...,14,855,20904122028,3.23,20904122031,20904122028,145.062231,-37.664970,145.087302,-37.659741
4,Y12H0000107P02T01,Car,Vehicle Passenger,Social,Social,20,20,Sunday,27/05/2012,May,...,10,630,20904122028,2.60,20904122028,20904122002,145.087302,-37.659741,145.071523,-37.651478


In [8]:
# Get detailed start time of trips
trips['STARTMIN'] = trips['STARTIME'] % 60
trips['START_DETAIL_TIME'] = (trips['TRAVDATE'].astype(str) + " " + trips['STARTHOUR'].astype(str) 
                              + ":" + trips['STARTMIN'].astype(str))

In [9]:
trips["Mode"].value_counts()

Walk     17115
Car      15764
Train     1704
Cycle     1704
Name: Mode, dtype: int64

In [10]:
trips.head().T

,0,1,2,3,4
TRIPID,Y12H0000107P01T01,Y12H0000107P01T02,Y12H0000107P01T03,Y12H0000107P01T04,Y12H0000107P02T01
Mode,Car,Car,Walk,Walk,Car
LINKMODE,Vehicle Driver,Vehicle Driver,Walking,Walking,Vehicle Passenger
TRIPPURP,Social,Social,Recreational,Recreational,Social
DESTPURP1,Social,At or Go Home,Recreational,At or Go Home,Social
TRIPTIME,20,10,30,30,20
TRAVTIME,20,10,30,30,20
TRAVDOW,Sunday,Sunday,Sunday,Sunday,Sunday
TRAVDATE,27/05/2012,27/05/2012,27/05/2012,27/05/2012,27/05/2012
TRAVMONTH,May,May,May,May,May


In [11]:
trips.loc[1,].ORIG_X, trips.loc[1,].ORIG_Y

(145.07152265574226, -37.65147795235493)

In [12]:
trips.loc[1,].DEST_X, trips.loc[1,].DEST_Y

(145.08730214277458, -37.659740939580466)

In [17]:
result = ls.car_route(
    origin=[trips.loc[1,].ORIG_Y, trips.loc[1,].ORIG_X],
    destination=[trips.loc[1,].DEST_Y, trips.loc[1,].DEST_X],
    return_results=[ ROUTING_RETURN.summary ],
)

In [18]:
print(result)

{"routes": [{"id": "5151367e-ca2c-4284-b3ed-0dddb9839c62", "sections": [{"arrival": {"place": {"location": {"lat": -37.659776, "lng": 145.0872978}, "originalLocation": {"lat": -37.659741, "lng": 145.0873021}, "type": "place"}, "time": "2021-08-06T09:42:39+10:00"}, "departure": {"place": {"location": {"lat": -37.6519, "lng": 145.07107}, "originalLocation": {"lat": -37.651478, "lng": 145.0715226}, "type": "place"}, "time": "2021-08-06T09:36:02+10:00"}, "id": "93bcc1c8-baf7-4c14-bfa0-d0ba6f3920a3", "summary": {"baseDuration": 379, "duration": 397, "length": 2386}, "transport": {"mode": "car"}, "type": "vehicle"}]}]}


In [19]:
datetime.strptime("27/05/2012 23:30",'%d/%m/%Y %H:%M')

datetime.datetime(2012, 5, 27, 23, 30)

In [20]:
def get_datetime(str_time):
    return datetime.strptime(str_time,'%d/%m/%Y %H:%M')

In [43]:
30000/60/60

8.333333333333334

In [30]:
def get_trip_los(trip):
    time.sleep(0.2)
    start_time = get_datetime(trip['START_DETAIL_TIME'])
    orig = [trip.ORIG_Y, trip.ORIG_X]
    dest = [trip.DEST_Y, trip.DEST_X]
    (trip['CAR_TIME'], trip['CAR_LEN'], trip['CYCLE_TIME'], trip['CYCLE_LEN'], 
     trip['WALK_TIME'], trip['WALK_LEN']) = collect_here_los(ls, orig, dest, start_time)
    return trip

In [44]:
trips.Mode.value_counts()

Walk     17115
Car      15764
Train     1704
Cycle     1704
Name: Mode, dtype: int64

In [48]:
# Divide data into walk_trips and not_walk_trips for smaller APIs number of queries
walk_trips = trips[trips.Mode.isin(['Walk'])]
no_w_trips = trips[~trips.Mode.isin(['Walk'])]
walk_trips.shape, no_w_trips.shape, trips.shape

((17115, 23), (19172, 23), (36287, 23))

In [ ]:
for gr, df in walk_trips.groupby()

In [49]:
batch_size = 100
np.arange(len(walk_trips)//batch_size)

NameError: name 'np' is not defined

In [38]:
%%time
trip_los = trips.apply(lambda x: get_trip_los(x), axis=1)
trip_los.shape

ApiError: 429, Too Many Requests, {"error":"Too Many Requests","error_description":"Rate limit for this service has been reached"}


In [40]:
trip_los.shape

(5, 29)

In [32]:
trip_los.head(10)

,TRIPID,Mode,LINKMODE,TRIPPURP,DESTPURP1,TRIPTIME,TRAVTIME,TRAVDOW,TRAVDATE,TRAVMONTH,...,DEST_X,DEST_Y,STARTMIN,START_DETAIL_TIME,CAR_TIME,CAR_LEN,CYCLE_TIME,CYCLE_LEN,WALK_TIME,WALK_LEN
0,Y12H0000107P01T01,Car,Vehicle Driver,Social,Social,20,20,Sunday,27/05/2012,May,...,145.071523,-37.651478,30,27/05/2012 10:30,382,2354,709,2364,2422,2332
1,Y12H0000107P01T02,Car,Vehicle Driver,Social,At or Go Home,10,10,Sunday,27/05/2012,May,...,145.087302,-37.659741,30,27/05/2012 12:30,379,2386,638,2386,2417,2332
2,Y12H0000107P01T03,Walk,Walking,Recreational,Recreational,30,30,Sunday,27/05/2012,May,...,145.062231,-37.664970,30,27/05/2012 13:30,421,3988,1014,3727,3743,3597
3,Y12H0000107P01T04,Walk,Walking,Recreational,At or Go Home,30,30,Sunday,27/05/2012,May,...,145.087302,-37.659741,15,27/05/2012 14:15,420,3971,964,3651,3749,3596
4,Y12H0000107P02T01,Car,Vehicle Passenger,Social,Social,20,20,Sunday,27/05/2012,May,...,145.071523,-37.651478,30,27/05/2012 10:30,382,2354,709,2364,2422,2332


In [35]:
trip_los.T

,0,1,2,3,4
TRIPID,Y12H0000107P01T01,Y12H0000107P01T02,Y12H0000107P01T03,Y12H0000107P01T04,Y12H0000107P02T01
Mode,Car,Car,Walk,Walk,Car
LINKMODE,Vehicle Driver,Vehicle Driver,Walking,Walking,Vehicle Passenger
TRIPPURP,Social,Social,Recreational,Recreational,Social
DESTPURP1,Social,At or Go Home,Recreational,At or Go Home,Social
TRIPTIME,20,10,30,30,20
TRAVTIME,20,10,30,30,20
TRAVDOW,Sunday,Sunday,Sunday,Sunday,Sunday
TRAVDATE,27/05/2012,27/05/2012,27/05/2012,27/05/2012,27/05/2012
TRAVMONTH,May,May,May,May,May


In [28]:
pd.DataFrame(trip_los)

,0
0,"[382, 2354, 709, 2364, 2422, 2332]"
1,"[379, 2386, 638, 2386, 2417, 2332]"
2,"[421, 3988, 1014, 3727, 3743, 3597]"
3,"[420, 3971, 964, 3651, 3749, 3596]"
4,"[382, 2354, 709, 2364, 2422, 2332]"


In [26]:
trips.head()

,TRIPID,Mode,LINKMODE,TRIPPURP,DESTPURP1,TRIPTIME,TRAVTIME,TRAVDOW,TRAVDATE,TRAVMONTH,...,HomeSA1,CUMDIST,ORIGSA1,DESTSA1,ORIG_X,ORIG_Y,DEST_X,DEST_Y,STARTMIN,START_DETAIL_TIME
0,Y12H0000107P01T01,Car,Vehicle Driver,Social,Social,20,20,Sunday,27/05/2012,May,...,20904122028,2.60,20904122028,20904122002,145.087302,-37.659741,145.071523,-37.651478,30,27/05/2012 10:30
1,Y12H0000107P01T02,Car,Vehicle Driver,Social,At or Go Home,10,10,Sunday,27/05/2012,May,...,20904122028,2.60,20904122002,20904122028,145.071523,-37.651478,145.087302,-37.659741,30,27/05/2012 12:30
2,Y12H0000107P01T03,Walk,Walking,Recreational,Recreational,30,30,Sunday,27/05/2012,May,...,20904122028,3.23,20904122028,20904122031,145.087302,-37.659741,145.062231,-37.664970,30,27/05/2012 13:30
3,Y12H0000107P01T04,Walk,Walking,Recreational,At or Go Home,30,30,Sunday,27/05/2012,May,...,20904122028,3.23,20904122031,20904122028,145.062231,-37.664970,145.087302,-37.659741,15,27/05/2012 14:15
4,Y12H0000107P02T01,Car,Vehicle Passenger,Social,Social,20,20,Sunday,27/05/2012,May,...,20904122028,2.60,20904122028,20904122002,145.087302,-37.659741,145.071523,-37.651478,30,27/05/2012 10:30


In [36]:
trips.shape

(36287, 23)